# Implementación de un lote de puntos de conexión

Imagine que en un centro médico se examina a los pacientes durante todo el día y los detalles de cada uno se guardan en un archivo independiente. Después, por la noche, el modelo de predicción de diabetes se puede usar para procesar todos los datos de pacientes del día como un lote, y generar predicciones que esperarán a que la mañana siguiente en el centro se pueda realizar el seguimiento de los pacientes de los que se ha predicho que pueden padecer diabetes. Con Azure Machine Learning, puede hacerlo mediante la creación de un punto de conexión por lotes; y eso es lo que implementará en este ejercicio.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Registro del modelo

Las implementaciones por lotes solo pueden implementar modelos registrados en el área de trabajo. Registrará un modelo de MLflow, que se almacena en la carpeta local `model`. 

In [ ]:
## Creación de un punto de conexión por lotes

Un punto de conexión por lotes es un punto de conexión HTTPS al que las aplicaciones pueden llamar para activar un trabajo de puntuación por lotes. El nombre de un punto de conexión por lotes debe ser único dentro de una región Azure. Usará la función `datetime` para generar un nombre único basado en la fecha y hora actuales. 

In [ ]:
Para crear un punto de conexión con la clase `BatchEndpoint`, debe especificar el nombre y, opcionalmente, una descripción. Después de crear un punto de conexión, implementará un modelo en el punto de conexión.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que el punto de conexión se cree antes de continuar. Debería aparecer una notificación verde en Studio. </p>

## Creación de la implementación

Una implementación es un conjunto de recursos necesarios para hospedar el modelo que realiza la inferencia real. Crearemos una implementación para nuestro punto de conexión mediante la clase `BatchDeployment`. 

Puesto que va a implementar un modelo de MLflow, no necesita un script de puntuación ni defina el entorno. Azure Machine Learning creará automáticamente esos recursos. El archivo `MLmodel` de la carpeta `model` se utiliza para comprender cuáles son las entradas y salidas previstas del modelo.

Implementará un modelo con los parámetros siguientes:

- `name`: nombre de la implementación.
- `description`: descripción opcional para aclarar aún más lo que representa la implementación.
- `endpoint_name`: nombre del punto de conexión creado anteriormente en el que se debe implementar el modelo.
- `model`: nombre del modelo registrado.
- `compute`: proceso que se va a usar al invocar el modelo implementado para generar predicciones.
- `instance_count`: recuento de nodos de proceso que se van a usar para generar predicciones.
- `max_concurrency_per_instance`: número máximo de ejecuciones de script de puntuación en paralelo por nodo de proceso.
- `mini_batch_size`: número de archivos pasados en cada ejecución del script de puntuación.
- `output_action`: cada nueva predicción se anexará como una nueva fila al archivo de salida.
- `output_file_name`: archivo al que se anexarán las predicciones.
- `retry_settings`: se produce un error en la configuración de un miniproceso por lotes.
- `logging_level`: el nivel de detalle del registro. Los valores permitidos son `warning`, `info` y `debug`. 

La ejecución de la celda siguiente configurará y creará la implementación.

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que se complete la implementación antes de continuar. Debería aparecer una notificación verde en Studio. </p>

Puede implementar varios modelos en un punto de conexión. Puede establecer la implementación predeterminada para especificar qué modelo se debe usar de forma predeterminada al llamar a un punto de conexión por lotes.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que se establezca la implementación predeterminada antes de continuar. Debería aparecer una notificación verde en Studio. </p>

## Preparación de los datos para las predicciones por lotes

En la carpeta `data` encontrará archivos CSV con datos sin etiquetar. Creará un recurso de datos que apunte a los archivos de la carpeta `data`, que usará como entrada para el trabajo por lotes.

In [ ]:
## Enviar el archivo

Ahora que ha implementado un modelo en un punto de conexión por lotes y tiene un recurso de datos sin etiquetar, está listo para invocar el punto de conexión para generar predicciones en los datos sin etiquetar.

En primer lugar, definirá la entrada haciendo referencia al recurso de datos registrado. A continuación, invocará el punto de conexión, que enviará un trabajo de canalización. Puede usar la dirección URL del trabajo para supervisarla en Studio. El trabajo contendrá un trabajo secundario que representa la ejecución del script de puntuación (generado) para obtener las predicciones.

## Obtención de los resultados

Cuando se complete el trabajo de canalización que invoca el punto de conexión por lotes, puede ver los resultados. Todas las predicciones se recopilan en el archivo `predictions.csv` almacenado en el almacén de datos predeterminado. Puede descargar el archivo y visualizar los datos ejecutando las celdas siguientes. 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score